In [1]:
!unzip /content/drive-download-20220416T184043Z-001.zip

Archive:  /content/drive-download-20220416T184043Z-001.zip
  inflating: subject_001_04__y.csv   
  inflating: subject_001_05__y_time.csv  
  inflating: subject_001_03__y.csv   
  inflating: subject_001_05__y.csv   
  inflating: subject_002_05__y_time.csv  
  inflating: subject_001_01__y.csv   
  inflating: subject_001_02__y_time.csv  
  inflating: subject_001_04__x_time.csv  
  inflating: subject_001_06__y.csv   
  inflating: subject_001_06__x_time.csv  
  inflating: subject_001_02__y.csv   
  inflating: subject_002_02__y_time.csv  
  inflating: subject_001_01__y_time.csv  
  inflating: subject_001_08__y_time.csv  
  inflating: subject_002_01__y.csv   
  inflating: subject_002_03__y.csv   
  inflating: subject_001_07__x_time.csv  
  inflating: subject_001_07__y_time.csv  
  inflating: subject_001_06__y_time.csv  
  inflating: subject_001_07__y.csv   
  inflating: subject_001_03__x_time.csv  
  inflating: subject_001_08__y.csv   
  inflating: subject_001_03__y_time.csv  
  inflating: su

In [34]:
import csv
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import imblearn
import warnings
warnings.filterwarnings('ignore')

In [35]:
def upsample_label(temp_x, temp_y):
    y_time_list = temp_y['time'].values
    y_label = temp_y['label'].values
    jdx = 0
    label_list = []
    for index,row in temp_x.iterrows():
        try:
            if row['time'] > y_time_list[jdx]:
                jdx+=1
            label_list.append(y_label[jdx])
        except:
            label_list.append(y_label[jdx - 1])
    return label_list

In [36]:
filepath = '/content/'
column_list = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z','subject', 'time', 'label']
final_df = pd.DataFrame([], columns = column_list)
for i in glob.glob(filepath + '*.csv'):
    print(i)
    df_type = i.split('.')[0].split('__')[1]
    file_name = i.split('/')[2].split('__')[0]
    if df_type == 'x':
        x = open(filepath + file_name + '__x.csv')
        x = pd.read_csv(x)
        x['subject'] = [file_name] * x.shape[0]
        
        x_time = open(filepath + file_name + '__x_time.csv')
        x_time = pd.read_csv(x_time)
        x['time'] = x_time
        
        y = open(filepath + file_name + '__y.csv')
        y = pd.read_csv(y)
        y['subject'] = [file_name] * y.shape[0]
        
        y_time = open(filepath + file_name + '__y_time.csv')
        y_time = pd.read_csv(y_time)
        y['time'] = y_time
        
        y.columns = ['label', 'subject', 'time']
        label_list = upsample_label(x, y)
        x['label'] = label_list
        x.columns = column_list
        final_df = pd.concat([final_df, x], ignore_index=True)

/content/subject_001_04__y_time.csv
/content/subject_001_08__y_time.csv
/content/subject_004_02__x_time.csv
/content/subject_001_06__x.csv
/content/subject_006_02__y_time.csv
/content/subject_002_05__x.csv
/content/subject_006_01__y_time.csv
/content/subject_002_02__y_time.csv
/content/subject_001_04__x_time.csv
/content/subject_005_02__y_time.csv
/content/subject_007_02__y.csv
/content/subject_005_02__x_time.csv
/content/subject_003_03__x.csv
/content/subject_003_03__y.csv
/content/subject_003_02__y.csv
/content/subject_002_03__x.csv
/content/subject_004_02__y_time.csv
/content/subject_006_01__y.csv
/content/subject_002_04__x.csv
/content/subject_001_07__x_time.csv
/content/subject_003_02__x.csv
/content/subject_002_01__y.csv
/content/subject_005_03__x_time.csv
/content/subject_002_03__y_time.csv
/content/subject_007_03__x.csv
/content/subject_007_04__x_time.csv
/content/subject_003_01__x_time.csv
/content/subject_001_04__x.csv
/content/subject_007_03__x_time.csv
/content/subject_004_

In [37]:
final_df.shape
from scipy import stats

def create_windows(X, y, time_steps=1, step=1):
    Xs, ys = [], []
      
    for i in range(0, len(X) - time_steps + 1, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)
        ys.append(stats.mode(labels)[0][0])

    return np.array(Xs), np.array(ys).reshape(-1, 1)

In [38]:
X = final_df[['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']]
y = final_df['label']

TIME_STEPS = 40  # Window Size
STEP = 1

X_data, y_data = create_windows(X, y, time_steps=TIME_STEPS,step=STEP)


In [39]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

df_label = pd.DataFrame(y_data, columns=['label'])
df_label['index_'] = df_label.index

## Library for performing undersampling 
rus = RandomUnderSampler(sampling_strategy='not minority', random_state=1)
df_balanced, balanced_labels = rus.fit_resample(df_label, df_label['label'])

sos = SMOTE(sampling_strategy='not majority')
df_balanced, balanced_labels  = sos.fit_resample(df_balanced, balanced_labels)

df_balanced = pd.DataFrame(df_balanced, columns=['label', 'index_'])

X_data = X_data[df_balanced['index_'].values]
y_data = y_data[df_balanced['index_'].values]

In [40]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.3, random_state=42)

## Converting label to OneHot Encoding
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
enc = enc.fit(y_train)
y_train = enc.transform(y_train)
y_val = enc.transform(y_val)

In [41]:
X_train = np.asarray(X_train).astype('float32')
X_val = np.asarray(X_val).astype('float32')

In [42]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from tensorflow.keras.layers import LSTM, LSTM, SimpleRNN, GRU, Bidirectional, Conv1D, Reshape
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import to_categorical

input_shape = (X_train.shape[1], X_train.shape[2])
output_dim = 4
lr = 0.001




In [47]:
model=Sequential()

model.add(Conv1D(32, 3, activation='relu',padding = "same",input_shape=input_shape))
model.add(Dropout(0.2)) 
model.add(Conv1D(64, 3, activation='relu',padding = "same"))
model.add(Dropout(0.2)) 
# model.add(Conv1D(128, 3, activation='relu',padding = "same"))
model.add(LSTM(256,return_sequences=True))
model.add(Dropout(0.5)) 
model.add(LSTM(256,return_sequences=False))
# model.add(LSTM(256, return_sequences=True))
# model.add(Dropout(0.5))
# model.add(LSTM(256, return_sequences=False))
# model.add(Dropout(0.5))
model.add(Dense(output_dim))
model.add(Activation("softmax"))
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [48]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_21 (Conv1D)          (None, 40, 32)            608       
                                                                 
 dropout_30 (Dropout)        (None, 40, 32)            0         
                                                                 
 conv1d_22 (Conv1D)          (None, 40, 64)            6208      
                                                                 
 dropout_31 (Dropout)        (None, 40, 64)            0         
                                                                 
 lstm_17 (LSTM)              (None, 40, 256)           328704    
                                                                 
 dropout_32 (Dropout)        (None, 40, 256)           0         
                                                                 
 lstm_18 (LSTM)              (None, 256)             

In [ ]:
hist = model.fit(
    X_train, y_train,
    validation_data = (X_val, y_val),
    epochs=20,
    batch_size=64,
    shuffle=True
)

Epoch 1/20
2406/2406 [==============================] - 89s 13ms/step - loss: 0.3190 - val_loss: 0.2011
Epoch 2/20
2406/2406 [==============================] - 28s 12ms/step - loss: 0.1936 - val_loss: 0.1686
Epoch 3/20
2406/2406 [==============================] - 29s 12ms/step - loss: 0.1586 - val_loss: 0.1157
Epoch 4/20
2406/2406 [==============================] - 28s 12ms/step - loss: 0.1382 - val_loss: 0.1130
Epoch 5/20
2406/2406 [==============================] - 29s 12ms/step - loss: 0.1239 - val_loss: 0.0954
Epoch 6/20
2406/2406 [==============================] - 29s 12ms/step - loss: 0.1126 - val_loss: 0.0961
Epoch 7/20
 596/2406 [======>.......................] - ETA: 18s - loss: 0.1081

In [14]:
y_pred = model.predict(X_val)
y_pred = np.argmax(y_pred, axis = 1)
y_val_ = np.argmax(y_val, axis = 1)

In [15]:
from sklearn.metrics import classification_report

print(classification_report(y_val_, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.95      0.96     16405
           1       0.99      1.00      1.00     16470
           2       0.99      1.00      0.99     16422
           3       0.97      0.97      0.97     16683

    accuracy                           0.98     65980
   macro avg       0.98      0.98      0.98     65980
weighted avg       0.98      0.98      0.98     65980

